In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras
import math
import numpy as np

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#top_model_weights_path = '../model/bottleneck_fc_model.h5'
top_model_weights_path = '../model/bottleneck_fc_model_inceptionv3_500_val_acc_4791.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 500, 500

#full
#train_data_dir = '../data/Pandora18K_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
#test_data_dir = '../data/Pandora18K_train_val_test_split/test'
#nb_train_samples = 14313
#nb_validation_samples = 1772
#total_num_classes = 18

#small
train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
nb_train_samples = 1462
nb_validation_samples = 167
total_num_classes = 18

#verysmall
#train_data_dir = '../data/verysmall/train'
#validation_data_dir = '../data/verysmall/val'
#nb_train_samples = 144 #157
#nb_validation_samples = 16 #18
#total_num_classes = 2

epochs = 2
batch_size = 32

Using TensorFlow backend.


In [2]:

# build the VGG16 network
base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
#base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.6))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(.6))
top_model.add(Dense(18, activation='softmax'))

top_model.load_weights(top_model_weights_path)

model = Model(input= base_model.input, output= top_model(base_model.output))

Model loaded.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


In [3]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

if False:
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                  metrics=['accuracy'])
    _, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = nb_validation_samples // batch_size)
    print("sanity_acc = " + str(sanity_acc))


Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Found 171 images belonging to 18 classes.


In [4]:
def run_with_parameters(lr):
    global best_acc
    global best_weight_path
    print("best val acc so far = " + str(best_acc))
    
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    #    for layer in model.layers:
    #        layer.trainable = True;
    frozen_layers = 25
    momentum = 0.9

    for layer in model.layers[:311]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr),
                  metrics=['accuracy'])
    
    pretrained_model = "temp_name"
    weight_dir = "../model/%s"%pretrained_model
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    param_names = "frozen_layers_" + str(frozen_layers) + "_lr_" + str(lr) + "_momentum_" + str(momentum)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)

    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpointer])
    
    model.load_weights(weight_path)
    _, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = nb_validation_samples // batch_size)
    print("sanity_acc = "+ str(sanity_acc))
    
    if sanity_acc > best_acc:
        model.load_weights(weight_path)
        model.save_weights(best_weight_path, overwrite = True)
        best_acc = sanity_acc
    else:
        os.remove(weight_path)
    
    print("best val acc so far = " + str(best_acc))

In [5]:
pretrained_model = "part_3_weights"
weight_dir = "../model/%s"%pretrained_model
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)

best_weight_path = "../model/best.h5"
best_acc = 0
best_lr = 0
best_frozen_layers = 0
    
for i in range(2):
    print("\n")
    
    lr = 10**np.random.uniform(low=-6, high=-3)
    print("lr = " + str(lr))

    for layer in model.layers[:best_frozen_layers]: #312 layers total
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr),
                  #momentum=0.9,
                  metrics=['accuracy'])
    
    param_names = "lr_" + str(lr)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)

    model.fit_generator(
        train_generator,
        steps_per_epoch=(nb_train_samples // batch_size)+1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=(nb_validation_samples // batch_size)+1,
        callbacks=[checkpointer])
    
    model.load_weights(weight_path)
    new_loss, new_acc = model.evaluate_generator(generator = validation_generator, steps = (nb_validation_samples // batch_size)+1)
    print("new_acc = "+ str(new_acc))
    print("new_loss = "+ str(new_loss))
    
    if new_acc > best_acc:
        model.load_weights(weight_path)
        model.save_weights(best_weight_path, overwrite = True)
        best_acc = new_acc
        best_lr = lr
    
    print("best val acc so far = " + str(best_acc))
    



lr = 2.18817300175e-05
Epoch 1/1
46/46 [==============================] - 83s - loss: 5.6024 - acc: 0.1730 - val_loss: 0.8567 - val_acc: 0.7665
new_acc = 0.772455090534
new_loss = 0.778284083583
best val acc so far = 0.772455090534


lr = 0.0106386074266
Epoch 1/1
46/46 [==============================] - 74s - loss: 2.7789 - acc: 0.1491 - val_loss: 1.8811 - val_acc: 0.3892
new_acc = 0.413173653408
new_loss = 1.8401535364
best val acc so far = 0.772455090534


In [6]:
if False:
    model.load_weights(best_weight_path)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=best_lr),
                  #momentum=0.9,
                  metrics=['accuracy'])

    sanity_loss, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = (nb_validation_samples // batch_size)+1)
    print("best val loss check = " + str(sanity_loss))
    print("best val acc check = " + str(sanity_acc))

best val loss check = 0.820405879064
best val acc check = 0.790419164532
